In [12]:
# --------------------------------------------------------------
# Secure Latent Impersonation Test
# MSc Dissertation - Defensive Validation Script
# --------------------------------------------------------------

# --------------------------------------------------------------
# LOGGING SETUP
# --------------------------------------------------------------

import logging
import os

LOG_DIR = os.path.join(os.getcwd(), "logs")
os.makedirs(LOG_DIR, exist_ok=True)

LOG_FILE = os.path.join(LOG_DIR, "biometric_auth_security_log.txt")

logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
)

def log_info(msg):
    logging.info(msg)
    print(msg)

def log_warning(msg):
    logging.warning(msg)
    print("WARNING:", msg)

def log_error(msg):
    logging.error(msg)
    print("ERROR:", msg)

print("Logging system initialised.")


log_info("--------------------------------------------------")
log_info("Secure Latent Impersonation Defensive Test Started")
log_info("--------------------------------------------------")

Logging system initialised.
--------------------------------------------------
Secure Latent Impersonation Defensive Test Started
--------------------------------------------------


In [13]:

import os
import sys
import time
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import logging

from PIL import Image
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1

In [14]:
# --------------------------------------------------------------
# CONFIGURATION (FAST TEST MODE)
# --------------------------------------------------------------

STYLEGAN_REPO = "/Users/stel/Documents/Dissertation/msc-biometric-security-clean/stylegan2-ada-pytorch"
STYLEGAN_PKL  = "/Users/stel/Documents/Dissertation/msc-biometric-security-clean/Models/stylegan2-ffhq.pkl"
TARGET_A_PATH = "/Users/stel/Documents/Dissertation/msc-biometric-security-clean/Datasets/lfw/Lenny_Kravitz/Lenny_Kravitz_0001.jpg"

LOW_THRESHOLD  = 0.80
HIGH_THRESHOLD = 0.90
MAX_ATTEMPTS   = 5

MAX_STEPS = 8      # FAST TEST (not 30 or 80)
LR = 0.03
TARGET_SIM = 0.85

In [15]:
# --------------------------------------------------------------
# DEVICE
# --------------------------------------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [16]:
# --------------------------------------------------------------
# FACENET
# --------------------------------------------------------------

facenet = InceptionResnetV1(pretrained="vggface2").eval().to(device)

to_facenet = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])

def load_image(path):
    return Image.open(path).convert("RGB")

def img_to_tensor(img):
    return to_facenet(img).unsqueeze(0).to(device)

def facenet_embed(img_tensor):
    emb = facenet(img_tensor)
    return nn.functional.normalize(emb, p=2, dim=1)


def cosine(a, b):
    return (a * b).sum(dim=1)
for param in facenet.parameters():
    param.requires_grad = False


In [17]:
# --------------------------------------------------------------
# STYLEGAN LOAD
# --------------------------------------------------------------

if STYLEGAN_REPO not in sys.path:
    sys.path.append(STYLEGAN_REPO)

with open(STYLEGAN_PKL, "rb") as f:
    G = pickle.load(f)["G_ema"].to(device)
    G.eval()

for param in G.parameters():
    param.requires_grad = False


def synth_from_w_single(w):
    if w.ndim == 2:
        w_plus = w.unsqueeze(1).repeat(1, G.synthesis.num_ws, 1)
    elif w.ndim == 3:
        w_plus = w
    else:
        raise ValueError("Unexpected latent shape")

    img = G.synthesis(w_plus, noise_mode="const")
    img = (img + 1) / 2
    return img.clamp(0, 1)

In [18]:
# --------------------------------------------------------------
# LOAD TARGET
# --------------------------------------------------------------

target_img = load_image(TARGET_A_PATH)
target_t = img_to_tensor(target_img)

with torch.no_grad():
    e_target = facenet_embed(target_t)
    
    
log_info("Target embedding loaded.")

print("Target embedding ready.")




Target embedding loaded.
Target embedding ready.


In [19]:

# --------------------------------------------------------------
# SECURE VERIFICATION LOGIC
# --------------------------------------------------------------

attempt_counter = {}
similarity_history = {}

def secure_verify(probe_embedding, reference_embedding, user_id):

    similarity = cosine(probe_embedding, reference_embedding).item()

    attempt_counter[user_id] = attempt_counter.get(user_id, 0) + 1
    log_info(f"Verification attempt | user={user_id} | similarity={similarity:.4f}")

    if attempt_counter[user_id] > MAX_ATTEMPTS:
        log_warning(f"User {user_id} locked due to excessive attempts")
        return "LOCKED", similarity

    if user_id not in similarity_history:
        similarity_history[user_id] = []

    similarity_history[user_id].append(similarity)

    if len(similarity_history[user_id]) >= 3:
        last_three = similarity_history[user_id][-3:]
        if last_three[0] < last_three[1] < last_three[2]:
            log_warning(f"Monotonic similarity increase detected for user {user_id}")

    if similarity >= HIGH_THRESHOLD:
        return "ACCEPT", similarity
    elif similarity >= LOW_THRESHOLD:
        log_info(f"STEP-UP triggered | user={user_id}")
        return "STEP_UP_REQUIRED", similarity
    else:
        return "REJECT", similarity



In [20]:
# --------------------------------------------------------------
# RUN REDUCED LATENT ATTACK (FAST MODE)
# --------------------------------------------------------------

try:
    log_info("Starting reduced latent impersonation attack (fast mode)")
    start_time = time.time()

    torch.manual_seed(0)
    z = torch.randn(1, G.z_dim, device=device)


    w = G.mapping(z, None)

    w = w.clone().detach().requires_grad_(True)
    optimizer = optim.Adam([w], lr=LR)

    for step in range(MAX_STEPS):

        img = synth_from_w_single(w)
        img_160 = nn.functional.interpolate(img, (160,160), mode="bilinear", align_corners=False)
        emb = facenet_embed(img_160)

        sim_target = cosine(emb, e_target)
        loss = 1 - sim_target.mean()
        print("img.requires_grad:", img.requires_grad)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        log_info(f"Attack Step {step} | Cosine_target={sim_target.item():.4f}")

        if sim_target.item() >= TARGET_SIM:
            log_info("Target similarity reached.")
            break

    runtime = (time.time() - start_time) / 60
    log_info(f"Attack runtime: {runtime:.2f} minutes")
    
    # ----------------------------------------------------------
    # TEST SECURE VERIFICATION
    # ----------------------------------------------------------

    decision, similarity = secure_verify(emb, e_target, user_id="attacker")

    log_info(f"Final decision: {decision} | Final cosine: {similarity:.4f}")

    # Repeat attempts to trigger rate limiting
    for i in range(6):
        decision, similarity = secure_verify(emb, e_target, user_id="attacker")
        log_info(f"Repeated attempt {i+1} -> {decision}")

except Exception as e:
    log_error(f"Fatal error occurred: {str(e)}")
    raise

log_info("Test completed.")


Starting reduced latent impersonation attack (fast mode)
img.requires_grad: True
Attack Step 0 | Cosine_target=-0.0902
img.requires_grad: True
Attack Step 1 | Cosine_target=0.0685
img.requires_grad: True
Attack Step 2 | Cosine_target=0.1825
img.requires_grad: True
Attack Step 3 | Cosine_target=0.2554
img.requires_grad: True
Attack Step 4 | Cosine_target=0.3109
img.requires_grad: True
Attack Step 5 | Cosine_target=0.3689
img.requires_grad: True
Attack Step 6 | Cosine_target=0.4566
img.requires_grad: True
Attack Step 7 | Cosine_target=0.5472
Attack runtime: 52.52 minutes
Verification attempt | user=attacker | similarity=0.5472
Final decision: REJECT | Final cosine: 0.5472
Verification attempt | user=attacker | similarity=0.5472
Repeated attempt 1 -> REJECT
Verification attempt | user=attacker | similarity=0.5472
Repeated attempt 2 -> REJECT
Verification attempt | user=attacker | similarity=0.5472
Repeated attempt 3 -> REJECT
Verification attempt | user=attacker | similarity=0.5472
Repeat